In [1]:
from pyspark.sql import SparkSession

In [4]:
spark = (
    SparkSession
    .builder
    .appName("wh-facts-dimensions-tables-iceberg")
    .master("spark://b0f96e8e1cf0:7077")
    .config("spark.executor.cores", 1)
    .config("spark.cores.max", 6)
    .config("spark.executor.memory", "512M")
    .getOrCreate()
)

25/01/03 01:46:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
spark

In [8]:
%%sql

SHOW TABLES IN demo.SolarX_Raw_Transactions

namespace,tableName,isTemporary
SolarX_Raw_Transactions,home_power_readings,False
SolarX_Raw_Transactions,solar_panel,False
SolarX_Raw_Transactions,solar_panel_readings,False


# Create a new name-space/database for the warehouse

In [9]:
%%sql

CREATE DATABASE IF NOT EXISTS SolarX_WH

++
||
++
++

# Home Power Usage WH

## Investigate home raw date

In [10]:
%%sql

DESCRIBE SolarX_Raw_Transactions.home_power_readings

col_name,data_type,comment
timestamp,timestamp,None
15_minutes_interval,int,None
min_consumption_wh,float,None
max_consumption_wh,float,None
,,
# Partitioning,,
Part 0,days(timestamp),
Part 1,15_minutes_interval,


In [11]:
%%sql

SELECT * FROM SolarX_Raw_Transactions.home_power_readings LIMIT 5

timestamp,15_minutes_interval,min_consumption_wh,max_consumption_wh
2013-01-02 01:15:00,1,0.0012557294685393572,0.004856623709201813
2013-01-02 01:15:00.005000,1,0.001255729584954679,0.004856622777879238
2013-01-02 01:15:00.010000,1,0.0012557297013700008,0.004856622312217951
2013-01-02 01:15:00.015000,1,0.0012557298177853227,0.0048566218465566635
2013-01-02 01:15:00.020000,1,0.0012557299342006445,0.004856621380895376


In [3]:
%%sql

SELECT 
    timestamp,
    DATE(timestamp) as date,
    15_minutes_interval,
    min_consumption_wh,
    max_consumption_wh
FROM 
    SolarX_Raw_Transactions.home_power_readings 
LIMIT 5

25/01/02 22:52:46 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


timestamp,date,15_minutes_interval,min_consumption_wh,max_consumption_wh
2013-01-02 01:15:00,2013-01-02,1,0.0012557294685393572,0.004856623709201813
2013-01-02 01:15:00.005000,2013-01-02,1,0.001255729584954679,0.004856622777879238
2013-01-02 01:15:00.010000,2013-01-02,1,0.0012557297013700008,0.004856622312217951
2013-01-02 01:15:00.015000,2013-01-02,1,0.0012557298177853227,0.0048566218465566635
2013-01-02 01:15:00.020000,2013-01-02,1,0.0012557299342006445,0.004856621380895376


### Query low frequency data for wh

In [4]:
%%sql

SELECT
    CONCAT(DATE(timestamp), '--', 15_minutes_interval) as home_power_readings_id,
    DATE(timestamp) as date,
    15_minutes_interval,
    SUM(min_consumption_wh) as min_consumption_power_wh,
    SUM(max_consumption_wh) as max_consumption_power_wh
FROM 
    SolarX_Raw_Transactions.home_power_readings
WHERE 
    DAY(timestamp) = 1
GROUP BY 
    DATE(timestamp), 15_minutes_interval
SORT BY
    15_minutes_interval
LIMIT 10

home_power_readings_id,date,15_minutes_interval,min_consumption_power_wh,max_consumption_power_wh
2013-01-01--0,2013-01-01,0,210.4949002956273,1234.37793413084
2013-01-01--1,2013-01-01,1,210.90460421564057,1195.791982460767
2013-01-01--2,2013-01-01,2,211.31430813518818,1157.2060307911597
2013-01-01--3,2013-01-01,3,211.72401205578353,1118.620079122018
2013-01-01--4,2013-01-01,4,206.7439376907423,1116.9222173416056
2013-01-01--5,2013-01-01,5,196.37402515218128,1152.1128553207964
2013-01-01--6,2013-01-01,6,186.00411261827685,1187.3034932990558
2013-01-01--7,2013-01-01,7,175.6342000772711,1222.494131277781
2013-01-01--8,2013-01-01,8,175.57182770763757,1223.2657041028142
2013-01-01--9,2013-01-01,9,185.8171100311447,1189.6178293367848


In [32]:
%%sql

SELECT
    DATE_TRUNC('minute', timestamp) AS truncated_timestamp,
    DATE(timestamp) as date,
    SUM(min_consumption_wh) as min_consumption_power_wh,
    SUM(max_consumption_wh) as max_consumption_power_wh
FROM 
    SolarX_Raw_Transactions.home_power_readings
WHERE 
    DAY(timestamp) = 1
GROUP BY 
    DATE(timestamp), DATE_TRUNC('minute', timestamp)
SORT BY
    DATE_TRUNC('minute', timestamp)
LIMIT 10

truncated_timestamp,date,min_consumption_power_wh,max_consumption_power_wh
2013-01-01 01:00:00,2013-01-01,14.02024700876791,83.49231410492212
2013-01-01 01:01:00,2013-01-01,14.02206791518256,83.32082098629326
2013-01-01 01:02:00,2013-01-01,14.02388882136438,83.1493278676644
2013-01-01 01:03:00,2013-01-01,14.025709727895446,82.97783474903554
2013-01-01 01:04:00,2013-01-01,14.027530634077266,82.806341631338
2013-01-01 01:05:00,2013-01-01,14.029351540491916,82.63484851270914
2013-01-01 01:06:00,2013-01-01,14.031172446790151,82.46335539454594
2013-01-01 01:07:00,2013-01-01,14.032993352971971,82.29186227545142
2013-01-01 01:08:00,2013-01-01,14.034814259386621,82.1203691563569
2013-01-01 01:09:00,2013-01-01,14.036635165684856,81.9488760381937


## Home readings dimension and fact tables

### Home dimension

In [6]:
%%sql

DROP TABLE IF EXISTS SolarX_WH.dim_home PURGE

25/01/03 01:47:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [7]:
%%sql

CREATE TABLE SolarX_WH.dim_home(
    home_key                                   SMALLINT    NOT NULL,
    min_consumption_power_wh                   FLOAT       NOT NULL,
    max_consumption_power_wh                   FLOAT       NOT NULL,

    -- scd type2 for min_consumption_power_wh
    min_consumption_power_wh_start_date        TIMESTAMP   NOT NULL,
    min_consumption_power_wh_end_date          TIMESTAMP,

    -- scd type2 for max_consumption_power_wh
    max_consumption_power_wh_start_date        TIMESTAMP  NOT NULL,
    max_consumption_power_wh_end_date          TIMESTAMP,

    current_flag                               BOOLEAN NOT NULL
)
USING iceberg;

++
||
++
++

In [44]:
%%sql

DROP TABLE IF EXISTS SolarX_WH.dim_home_appliances PURGE

25/01/03 00:29:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


++
||
++
++

In [45]:
%%sql

CREATE TABLE SolarX_WH.dim_home_appliances(
    home_appliance_key                  SMALLINT    NOT NULL,
    home_key                            SMALLINT    NOT NULL, -- REFERENCES dim_home(home_key)
    appliance                           VARCHAR(25) NOT NULL,    
    min_consumption_power_wh            FLOAT       NOT NULL,
    max_consumption_power_wh            FLOAT       NOT NULL,
    usage_time                          VARCHAR(50) NOT NULL
)
USING iceberg;

++
||
++
++

### Home fact

In [27]:
%%sql

DROP TABLE IF EXISTS SolarX_WH.fact_home_power_readings PURGE

++
||
++
++

In [28]:
%%sql

CREATE TABLE SolarX_WH.fact_home_power_readings(
    home_power_reading_key          INT           NOT NULL,
    home_key                        SMALLINT      NOT NULL,   -- REFERENCES dim_home(home_key)
    date_key                        TIMESTAMP     NOT NULL,   -- REFERENCES dim_date(date_key)

    home_power_reading_id           VARCHAR(25)   NOT NULL,
    15_minutes_interval             SMALLINT      NOT NULL,
    min_consumption_power_wh        FLOAT         NOT NULL,
    max_consumption_power_wh        FLOAT         NOT NULL 
)

USING iceberg
PARTITIONED BY (MONTH(date_key), 15_minutes_interval)

++
||
++
++

## Solar panel readings dimension and fact tables

## Investigate solar panel raw date

In [52]:
%%sql

DESCRIBE SolarX_Raw_Transactions.solar_panel

col_name,data_type,comment
id,int,None
name,string,None
capacity_kwh,float,None
intensity_power_rating,float,None
temperature_power_rating,float,None


In [53]:
%%sql

DESCRIBE SolarX_Raw_Transactions.solar_panel_readings

col_name,data_type,comment
timestamp,timestamp,None
15_minutes_interval,int,None
panel_id,int,None
generation_power_wh,float,None
,,
# Partitioning,,
Part 0,days(timestamp),
Part 1,panel_id,
Part 2,15_minutes_interval,


### Query low frequency data for wh

In [61]:
%%sql

SELECT
    CONCAT(DATE(timestamp), '--', 15_minutes_interval) as solar_panel_power_readings_id,
    DATE(timestamp) as date,
    15_minutes_interval,
    SUM(generation_power_wh) as generation_power_wh
FROM 
    SolarX_Raw_Transactions.solar_panel_readings
WHERE 
    DAY(timestamp) = 1 AND panel_id = 1
GROUP BY 
    DATE(timestamp), 15_minutes_interval
SORT BY
    generation_power_wh desc
LIMIT 10

solar_panel_power_readings_id,date,15_minutes_interval,generation_power_wh
2013-01-01--34,2013-01-01,34,1112.372815785464
2013-01-01--32,2013-01-01,32,1108.504030722659
2013-01-01--37,2013-01-01,37,1075.1697589475662
2013-01-01--33,2013-01-01,33,1070.1509305494837
2013-01-01--39,2013-01-01,39,1033.8308916147798
2013-01-01--35,2013-01-01,35,1033.8161300485954
2013-01-01--38,2013-01-01,38,1001.8857660298236
2013-01-01--36,2013-01-01,36,997.2686491240747
2013-01-01--42,2013-01-01,42,929.7826389158145
2013-01-01--41,2013-01-01,41,923.0105679007247


### Solar panel dimension

In [16]:
%%sql

DROP TABLE IF EXISTS SolarX_WH.dim_solar_panel PURGE

++
||
++
++

In [17]:
%%sql

CREATE TABLE SolarX_WH.dim_solar_panel(
    solar_panel_key                             SMALLINT    NOT NULL,
    solar_panel_id                              SMALLINT    NOT NULL,
    name                                        VARCHAR(20) NOT NULL,    
    capacity_kwh                                FLOAT       NOT NULL,
    intensity_power_rating_wh                   FLOAT       NOT NULL,
    temperature_power_rating_c                  FLOAT       NOT NULL,

    -- scd type2 for capacity_kwh
    capacity_kwh_start_date                     TIMESTAMP   NOT NULL,
    capacity_kwh_end_date                       TIMESTAMP,

    -- scd type2 for intensity_power_rating_wh
    intensity_power_rating_wh_start_date        TIMESTAMP  NOT NULL,
    intensity_power_rating_wh_end_date          TIMESTAMP,

    -- scd type2 for temperature_power_rating_c
    temperature_power_rating_c_start_date       TIMESTAMP  NOT NULL,
    temperature_power_rating_c_end_date         TIMESTAMP
)
USING iceberg;

++
||
++
++

### Solar panel fact

In [29]:
%%sql

DROP TABLE IF EXISTS SolarX_WH.fact_solar_panel_power_readings PURGE

++
||
++
++

In [30]:
%%sql

CREATE TABLE SolarX_WH.fact_solar_panel_power_readings(
    solar_panel_reading_key         INT           NOT NULL,
    solar_panel_key                 SMALLINT      NOT NULL,   -- REFERENCES dim_solar_panel(solar_panel_key)
    date_key                        TIMESTAMP     NOT NULL,   -- REFERENCES dim_date(date_key)

    solar_panel_reading_id          VARCHAR(25)   NOT NULL,
    15_minutes_interval             SMALLINT      NOT NULL,
    generation_power_wh             FLOAT         NOT NULL 
)

USING iceberg
PARTITIONED BY (MONTH(date_key), 15_minutes_interval)

++
||
++
++

## Date dimension table

In [8]:
%%sql

DROP TABLE IF EXISTS SolarX_WH.dim_date PURGE

++
||
++
++

In [10]:
%%sql

CREATE TABLE SolarX_WH.dim_date
(
    date_key            TIMESTAMP  NOT NULL,
    year                SMALLINT   NOT NULL,
    quarter             SMALLINT   NOT NULL,
    month               SMALLINT   NOT NULL,
    week                SMALLINT   NOT NULL,
    day                 SMALLINT   NOT NULL,
    hour                SMALLINT   NOT NULL,
    minute              SMALLINT   NOT NULL,
    is_weekend          BOOLEAN    NOT NULL
)

USING iceberg
PARTITIONED BY (month, minute)

++
||
++
++

In [8]:
spark.stop()